<h1> Developing the Feature Detection and Image Classifier <h1>

In [50]:
import pandas as pd
import cv2
import  requests
from io import BytesIO
from PIL import Image
import numpy as np

1.- <b> Data exploration from the bulk data from API <b> 
    
After the name and type of a card have been extracted, there will probably be less than 100 posibilities of mach. Here we will go through the process of implementing the json file data extraction, putting in into a dataframe and cleaning the data.

In [3]:
%time df = pd.read_json (r'C:\Users\nessa\Poromagia\Poromagia\back_end\resources\all-cards-20221006091839.json')
df
# columns=['id', 'name', 'type_line', 'image_status', 'image_uris', 'prices', 'scryfall_uri']

CPU times: total: 35.5 s
Wall time: 35.9 s


,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,produced_mana,card_faces,color_indicator,tcgplayer_etched_id,content_warning,attraction_lights,life_modifier,hand_modifier,variation_of,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,00009878-d086-46f0-a964-15734d8368ac,30cd69a8-7893-4075-94ca-04450ff821d3,[433932],NaN,NaN,NaN,NaN,Spirit of the Hearth,fr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000a54c-a511-4925-92dc-01b937f9afad,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,[],NaN,NaN,98659.0,NaN,Spirit,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383931,card,ffff1c55-ffcc-46ab-8187-55d0de1cc912,f01efb04-ad2b-46fb-852c-dfb7e523e1b7,[],NaN,NaN,NaN,NaN,"Mirri, Cat Warrior",fr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
383932,card,ffff1ee3-6e15-445b-83f9-9b76e46d3d5f,9bc82078-2f3e-44ee-906d-707f3430d414,[368413],NaN,NaN,NaN,NaN,Hands of Binding,ja,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
383933,card,ffff211d-cb68-4d9e-a944-f8757fe68e9e,87217160-0687-4482-b9a3-91bad8527675,[385096],NaN,NaN,NaN,NaN,Soul of Zendikar,ja,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
383934,card,ffff5100-6edb-4f7a-8d4d-7ab7cc188ca7,e7564d66-767c-4cd9-a5f0-0f2488a4a74b,[477540],NaN,NaN,NaN,NaN,Heliod's Intervention,ja,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df.head()

,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,produced_mana,card_faces,color_indicator,tcgplayer_etched_id,content_warning,attraction_lights,life_modifier,hand_modifier,variation_of,flavor_name
0,card,0000579f-7b35-4ed3-b44c-db2a538066fe,44623693-51d6-49ad-8cd7-140505caf02f,[109722],25527.0,25528.0,14240.0,13850.0,Fury Sliver,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,card,00006596-1166-4a79-8443-ca9f82e6db4e,8ae3562f-28b7-4462-96ed-be0cf7052ccc,[189637],34586.0,34587.0,33347.0,21851.0,Kor Outfitter,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,card,00009878-d086-46f0-a964-15734d8368ac,30cd69a8-7893-4075-94ca-04450ff821d3,[433932],NaN,NaN,NaN,NaN,Spirit of the Hearth,fr,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,card,0000a54c-a511-4925-92dc-01b937f9afad,dc4e2134-f0c2-49aa-9ea3-ebf83af1445c,[],NaN,NaN,98659.0,NaN,Spirit,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,card,0000cd57-91fe-411f-b798-646e965eec37,9f0d82ae-38bf-45d8-8cda-982b6ead1d72,[435231],65170.0,65171.0,145764.0,301766.0,Siren Lookout,en,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
keep_columns=['id', 'name', 'type_line', 'image_status', 'image_uris', 'scryfall_uri', 'lang']
df = df[keep_columns]
df

,id,name,type_line,image_status,image_uris,scryfall_uri,lang
0,0000579f-7b35-4ed3-b44c-db2a538066fe,Fury Sliver,Creature — Sliver,highres_scan,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/tsp/157/fury-sliver?...,en
1,00006596-1166-4a79-8443-ca9f82e6db4e,Kor Outfitter,Creature — Kor Soldier,highres_scan,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/zen/21/kor-outfitter...,en
2,00009878-d086-46f0-a964-15734d8368ac,Spirit of the Hearth,Creature — Cat Spirit,lowres,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/c17/73/fr/esprit-du-...,fr
3,0000a54c-a511-4925-92dc-01b937f9afad,Spirit,Token Creature — Spirit,highres_scan,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/tmm2/5/spirit?utm_so...,en
4,0000cd57-91fe-411f-b798-646e965eec37,Siren Lookout,Creature — Siren Pirate,highres_scan,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/xln/78/siren-lookout...,en
...,...,...,...,...,...,...,...
383931,ffff1c55-ffcc-46ab-8187-55d0de1cc912,"Mirri, Cat Warrior",Legendary Creature — Cat Warrior,placeholder,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/exo/114/fr/mirri-gue...,fr
383932,ffff1ee3-6e15-445b-83f9-9b76e46d3d5f,Hands of Binding,Sorcery,lowres,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/gtc/37/ja/%E6%9D%9F%...,ja
383933,ffff211d-cb68-4d9e-a944-f8757fe68e9e,Soul of Zendikar,Creature — Avatar,lowres,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/m15/201/ja/%E3%82%BC...,ja
383934,ffff5100-6edb-4f7a-8d4d-7ab7cc188ca7,Heliod's Intervention,Instant,lowres,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/thb/19/ja/%E3%83%98%...,ja


In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_columns = df.columns.tolist() #type(df_columns)
keep_columns=['id', 'name', 'type_line', 'image_status', 'image_uris', 'scryfall_uri', 'lang']
to_drop = []
for column in df_columns:
    if column not in keep_columns:
        to_drop.append(column)
print(to_drop)

[]


In [6]:
# df.drop(['column_1', 'Column_2'], axis = 1, inplace = True) 
df.drop(to_drop, axis = 1, inplace = True)
df_clean = df.copy()

In [7]:
print("the clean dataframe has",len(df_clean.index),"columns") #WAO!
#deleting the "," "-" " ' " as text parser does not recognize them
df_clean['type_line'] = df_clean['type_line'].replace("—|,|'", '', regex=True)
df_clean['name'] = df_clean['name'].replace("—|,|'", '', regex=True) 
df_clean['type_line'] = df_clean['type_line'].str.replace('  ', ' ')
df_clean['name'] = df_clean['name'].str.replace('  ', ' ')


df_clean.head()

the clean dataframe has 383936 columns


,id,name,type_line,image_status,image_uris,scryfall_uri,lang
0,0000579f-7b35-4ed3-b44c-db2a538066fe,Fury Sliver,Creature Sliver,highres_scan,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/tsp/157/fury-sliver?...,en
1,00006596-1166-4a79-8443-ca9f82e6db4e,Kor Outfitter,Creature Kor Soldier,highres_scan,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/zen/21/kor-outfitter...,en
2,00009878-d086-46f0-a964-15734d8368ac,Spirit of the Hearth,Creature Cat Spirit,lowres,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/c17/73/fr/esprit-du-...,fr
3,0000a54c-a511-4925-92dc-01b937f9afad,Spirit,Token Creature Spirit,highres_scan,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/tmm2/5/spirit?utm_so...,en
4,0000cd57-91fe-411f-b798-646e965eec37,Siren Lookout,Creature Siren Pirate,highres_scan,{'small': 'https://cards.scryfall.io/small/fro...,https://scryfall.com/card/xln/78/siren-lookout...,en


In [8]:
# the column 'iamge_uris' is a dictionary, contains the image of the card in "small","png","border_crop"... we need large
df_images = pd.json_normalize(df_clean['image_uris'])
# copy in order to avoid bugs
df_images = df_images['large'].copy()
# converting to df again as the result is an object
df_images = pd.DataFrame(data=df_images)
print("the df_images has",len(df_images.index),"columns") #so manyyy
df_images.head()

the df_images has 383936 columns


,large
0,https://cards.scryfall.io/large/front/0/0/0000...
1,https://cards.scryfall.io/large/front/0/0/0000...
2,https://cards.scryfall.io/large/front/0/0/0000...
3,https://cards.scryfall.io/large/front/0/0/0000...
4,https://cards.scryfall.io/large/front/0/0/0000...


In [9]:
# joining, renaming and rearranging columns
pd.set_option('display.max_colwidth', None)
df2 = pd.concat([df_clean, df_images],axis=1)
df2.drop(["image_uris"], inplace=True, axis=1)
df2.rename(columns = {'id':'card_ID', 'name':'card_name', 'type_line': 'card_type', 'large': 'card_image', 'scryfall_uri':'card_scryfall_URL', 'image_status': 'card_image_status', 'lang':'card_language'}, inplace = True)
df2.head()

,card_ID,card_name,card_type,card_image_status,card_scryfall_URL,card_language,card_image
0,0000579f-7b35-4ed3-b44c-db2a538066fe,Fury Sliver,Creature Sliver,highres_scan,https://scryfall.com/card/tsp/157/fury-sliver?utm_source=api,en,https://cards.scryfall.io/large/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979
1,00006596-1166-4a79-8443-ca9f82e6db4e,Kor Outfitter,Creature Kor Soldier,highres_scan,https://scryfall.com/card/zen/21/kor-outfitter?utm_source=api,en,https://cards.scryfall.io/large/front/0/0/00006596-1166-4a79-8443-ca9f82e6db4e.jpg?1562609251
2,00009878-d086-46f0-a964-15734d8368ac,Spirit of the Hearth,Creature Cat Spirit,lowres,https://scryfall.com/card/c17/73/fr/esprit-du-foyer?utm_source=api,fr,https://cards.scryfall.io/large/front/0/0/00009878-d086-46f0-a964-15734d8368ac.jpg?1562597860
3,0000a54c-a511-4925-92dc-01b937f9afad,Spirit,Token Creature Spirit,highres_scan,https://scryfall.com/card/tmm2/5/spirit?utm_source=api,en,https://cards.scryfall.io/large/front/0/0/0000a54c-a511-4925-92dc-01b937f9afad.jpg?1562701869
4,0000cd57-91fe-411f-b798-646e965eec37,Siren Lookout,Creature Siren Pirate,highres_scan,https://scryfall.com/card/xln/78/siren-lookout?utm_source=api,en,https://cards.scryfall.io/large/front/0/0/0000cd57-91fe-411f-b798-646e965eec37.jpg?1562549609


In [105]:
df2 = df2[['card_ID','card_name','card_type', 'card_language','card_image_status', 'card_image', 'card_scryfall_URL']]
df2.head()

,card_ID,card_name,card_type,card_language,card_image_status,card_image,card_scryfall_URL
0,0000579f-7b35-4ed3-b44c-db2a538066fe,Fury Sliver,Creature Sliver,en,highres_scan,https://cards.scryfall.io/large/front/0/0/0000579f-7b35-4ed3-b44c-db2a538066fe.jpg?1562894979,https://scryfall.com/card/tsp/157/fury-sliver?utm_source=api
1,00006596-1166-4a79-8443-ca9f82e6db4e,Kor Outfitter,Creature Kor Soldier,en,highres_scan,https://cards.scryfall.io/large/front/0/0/00006596-1166-4a79-8443-ca9f82e6db4e.jpg?1562609251,https://scryfall.com/card/zen/21/kor-outfitter?utm_source=api
2,00009878-d086-46f0-a964-15734d8368ac,Spirit of the Hearth,Creature Cat Spirit,fr,lowres,https://cards.scryfall.io/large/front/0/0/00009878-d086-46f0-a964-15734d8368ac.jpg?1562597860,https://scryfall.com/card/c17/73/fr/esprit-du-foyer?utm_source=api
3,0000a54c-a511-4925-92dc-01b937f9afad,Spirit,Token Creature Spirit,en,highres_scan,https://cards.scryfall.io/large/front/0/0/0000a54c-a511-4925-92dc-01b937f9afad.jpg?1562701869,https://scryfall.com/card/tmm2/5/spirit?utm_source=api
4,0000cd57-91fe-411f-b798-646e965eec37,Siren Lookout,Creature Siren Pirate,en,highres_scan,https://cards.scryfall.io/large/front/0/0/0000cd57-91fe-411f-b798-646e965eec37.jpg?1562549609,https://scryfall.com/card/xln/78/siren-lookout?utm_source=api


<b>Conclusion:<b> 
    
Cleaning has been done for now!
I have to find a way to transform a pandas dataframe to mysql database. Figure out how the database will be updated daily: fetching from API? downloading the bluk file & cleaning it everytime? if so, the process showed above should be an script?

<b>2. Image finding <b>
    
finding the url of images that have the same name and type as the image chosen to find the pair of. I will just be hardcoding the card_name and card_type as the text_parser class is the other document. Cards have different languages, and the library easyocr can exact text in multiple languages. I can extract the text from the description of the card, from there I get the language, and using TextBlob we can also get the exact language the card is written in. Therefore, we would have one cattegory more to reduce the possible matches.
https://stackoverflow.com/questions/39142778/how-to-determine-the-language-of-a-piece-of-text

In [110]:
#harcoding the name && type of one card to test it for now
card_name = "Kenriths Transformation"
card_type = "Enchantment Aura"
card_year = "2019"
card_language = "en"
card_path = r"C:\Users\nessa\Poromagia\Poromagia\back_end\resources\img\4.jpg"


In [9]:

#selectiong rows based on condition
%time df_possible = df2.loc[(df2['card_type'] == card_type) & (df2['card_name'] == card_name) & (df2['card_language'] == card_language)]
len(df_possible)

NameError: name 'df2' is not defined

NameError: name 'df_possible' is not defined

In [32]:
# what happens if i try to call the API?
# https://365datascience.com/tutorials/python-tutorials/limit-rate-requests-web-scraping/
import requests
import time
API = 'https://api.scryfall.com/cards/search?order=rarity&q=' 

In [111]:
name_api = f"{API}!'{card_name}'+include%3Aextras&unique=prints"
print(name_api)

https://api.scryfall.com/cards/search?order=rarity&q=!'Kenriths Transformation'+include%3Aextras&unique=prints


In [112]:
card_type_list = list(card_type.split(" "))
string = ""
for i in range(len(card_type_list)):
    string += f"t:{card_type_list[i]}"
    if i < len(card_type_list) -1:
        string += "+"

print(string)

t:Enchantment+t:Aura


In [113]:
type_year_lan_api = f"{API}{string}+year:{card_year}"
print(type_year_lan_api)

https://api.scryfall.com/cards/search?order=rarity&q=t:Enchantment+t:Aura+year:2019


In [109]:
type_year_lan_api = f"{API}{string}"
print(type_year_lan_api)

https://api.scryfall.com/cards/search?order=rarity&q=t:Instant


In [114]:
response_json = ""
result = 0
possible_matches = []
# 0 -> nothing worked! 
# 1 -> the name_api worked!
# 2 -> the type_year_lan_api worked!

def requests_to_server():
    
    response = requests.get(name_api)
    if response.status_code == 200:
        response_json = response.json()
        result = 1
    else:
        time.sleep(2) #sleeping two seconds to avoid getting banned!
        response = requests.get(type_year_lan_api)
        if response.status_code == 200:
            response_json = response.json()
            result = 2

    if result != 0:
        for each_card in response_json["data"]:
            possible_match = {}
            possible_match["card_id"] = each_card["id"]
            
            # converting image_link into an image_array
            try:
                image_url = each_card["image_uris"]["large"]
            except: # large image does not exist
                image_url = each_card["image_uris"]["normal"]
            original_img = Image.open(requests.get(image_url, stream=True).raw)
            img = np.asarray(original_img)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            possible_match["card_image"] = img            
            possible_matches.append(possible_match)
            
    
    return result

print(requests_to_server())
print(possible_matches)

1
[{'card_id': '44d60a41-3f5e-4559-b18c-22c0fe15235e', 'card_image': array([[254, 254, 254, ..., 254, 254, 254],
       [254, 254, 254, ..., 254, 254, 254],
       [254, 254, 254, ..., 254, 254, 254],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)}, {'card_id': '6da7cd39-1f8a-4f68-adb7-df2beac02263', 'card_image': array([[164, 211, 206, ..., 207, 211, 164],
       [211, 254, 254, ..., 254, 254, 212],
       [205, 254, 251, ..., 252, 254, 206],
       ...,
       [205, 254, 251, ..., 252, 255, 206],
       [211, 255, 254, ..., 255, 255, 212],
       [164, 211, 206, ..., 207, 211, 164]], dtype=uint8)}, {'card_id': '896e5f61-6074-458d-9e8c-aff99fda75d7', 'card_image': array([[254, 254, 254, ..., 254, 254, 254],
       [254, 254, 254, ..., 254, 254, 254],
       [254, 254, 254, ..., 254, 254, 254],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ...,

In [103]:
df_possible.head(5)

,card_ID,card_name,card_type,card_language,card_image_status,card_image,card_scryfall_URL
15279,0a14b17e-bfff-4859-92cb-a82d2e90580b,Hour of Reckoning,Sorcery,en,highres_scan,https://cards.scryfall.io/large/front/0/a/0a14b17e-bfff-4859-92cb-a82d2e90580b.jpg?1637629234,https://scryfall.com/card/mic/87/hour-of-reckoning?utm_source=api
85351,38c1c3ed-d344-440c-bec0-09ad682b2727,Hour of Reckoning,Sorcery,en,highres_scan,https://cards.scryfall.io/large/front/3/8/38c1c3ed-d344-440c-bec0-09ad682b2727.jpg?1541002870,https://scryfall.com/card/gk1/104/hour-of-reckoning?utm_source=api
108636,4854af5b-1771-4f1c-93e0-d107db28d97c,Hour of Reckoning,Sorcery,en,highres_scan,https://cards.scryfall.io/large/front/4/8/4854af5b-1771-4f1c-93e0-d107db28d97c.jpg?1592712781,https://scryfall.com/card/cmd/15/hour-of-reckoning?utm_source=api
212090,8d787021-6a02-4394-875d-ef529fa3d38e,Hour of Reckoning,Sorcery,en,highres_scan,https://cards.scryfall.io/large/front/8/d/8d787021-6a02-4394-875d-ef529fa3d38e.jpg?1663387480,https://scryfall.com/card/40k/187/hour-of-reckoning?utm_source=api
285937,bec7a987-1ef2-40aa-a744-92d90b246df4,Hour of Reckoning,Sorcery,en,highres_scan,https://cards.scryfall.io/large/front/b/e/bec7a987-1ef2-40aa-a744-92d90b246df4.jpg?1598913735,https://scryfall.com/card/rav/21/hour-of-reckoning?utm_source=api


In [13]:
# making a list out of all the posibilities of card matching
possible_images = df_possible['card_image'].tolist()
possible_ids = df_possible['card_ID'].tolist()
print("there are",len(possible_images), len(possible_ids),"posibilities of matches")
possible_images
# possible_ids[0:5]

there are 6 6 posibilities of matches


['https://cards.scryfall.io/large/front/0/a/0a14b17e-bfff-4859-92cb-a82d2e90580b.jpg?1637629234',
 'https://cards.scryfall.io/large/front/3/8/38c1c3ed-d344-440c-bec0-09ad682b2727.jpg?1541002870',
 'https://cards.scryfall.io/large/front/4/8/4854af5b-1771-4f1c-93e0-d107db28d97c.jpg?1592712781',
 'https://cards.scryfall.io/large/front/8/d/8d787021-6a02-4394-875d-ef529fa3d38e.jpg?1663387480',
 'https://cards.scryfall.io/large/front/b/e/bec7a987-1ef2-40aa-a744-92d90b246df4.jpg?1598913735',
 'https://cards.scryfall.io/large/front/d/1/d12768a5-8ee6-407b-87cf-703e69a0c32a.jpg?1568003844']

In [14]:
def collecting_arrays(possible_images: list) -> list:
    possible_arrays= []
    for image_url in possible_images:
        # getting the image frmo the url link from the df
        img = Image.open(requests.get(image_url, stream=True).raw)
        img.show()
        # converting it into black & into an array
        img =  np.asarray(img.convert('L'))
        # making the dictionary as card_id: card_image_array
        possible_arrays.append(img)
    return possible_arrays

In [35]:
def collecting_arrayss(possible_images: list) -> list:
    possible_arrays= []
    for image_url in possible_images:
        # getting the image frmo the url link from the df
        img = Image.open(requests.get(image_url, stream=True).raw)
        img = np.asarray(img)
        # converting it into black & into an array
        img = cv2.cvtColor( img , cv2.COLOR_BGR2GRAY)
        # Image.fromarray(img).show()
        # making the dictionary as card_id: card_image_array
        possible_arrays.append(img)
    return possible_arrays

In [36]:
possible_arrays = collecting_arrayss(possible_images)
possible_arrays = np.array(possible_arrays)
possible_arrays 

array([[[254, 254, 254, ..., 254, 254, 254],
        [254, 254, 254, ..., 254, 254, 254],
        [254, 254, 254, ..., 254, 254, 254],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[  7,   1,   1, ...,   6,   1,  11],
        [  1,   1,  13, ...,   2,   7,   1],
        [ 12,   1,   1, ...,   1,   1,  10],
        ...,
        [ 11,   1,   0, ...,   1,   1,   9],
        [  0,   4,   8, ...,   8,   4,   1],
        [  9,   0,   2, ...,   2,   1,  10]],

       [[254, 254, 254, ..., 254, 254, 254],
        [254, 254, 254, ..., 254, 254, 254],
        [254, 254, 254, ..., 254, 254, 254],
        ...,
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255],
        [255, 255, 255, ..., 255, 255, 255]],

       [[  4,  14,   4, ...,   4,   5,   4],
        [  5,   4,   5, ...,  10,   2,   4],
        [  4,   4,  19, ...,  15,   3,   3],
        ..

<b> 3. Classifier <b>

In [84]:
# trying to measure time & apply numba
from numba import jit, njit

In [99]:
### process of getting the image from the camera integrated into the raspberry
img1 = cv2.imread(card_path,0)
img1 = cv2.rotate(img1,cv2.ROTATE_90_CLOCKWISE)
Image.fromarray(img1).show()



In [19]:
## getting a random possible match to try the classifier
img2 = possible_arrays[1]
Image.fromarray(img2).show()

In [123]:
#Works well with images of different dimensions
def orb_sim(img1, img2):
  # SIFT is no longer available in cv2 so using ORB
  orb = cv2.ORB_create()

  # detect keypoints and descriptors
  kp_a, desc_a = orb.detectAndCompute(img1, None)
  kp_b, desc_b = orb.detectAndCompute(img2, None)

  # define the bruteforce matcher object
  bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
  #perform matches. 
  matches = bf.match(desc_a, desc_b)
  #Look for similar regions with distance < 50. Goes from 0 to 100 so pick a number between.
  similar_regions = [i for i in matches if i.distance < 55]  
  if len(matches) == 0:
    return 0
  return len(similar_regions) / len(matches)


In [21]:
orb_similarity = orb_sim(img1, img2)  #1.0 means identical. Lower = not similar
print("Similarity using ORB is: ", orb_similarity)

Similarity using ORB is:  0.40441176470588236


In [65]:
# making the loop to check through all the possible matches and not only one
def similarities_array(image: int, possible_arrays: list) -> int:
    similarities_percentages = []
    for array in possible_arrays:
        percentage = orb_sim(image, array)
        similarities_percentages.append(percentage)
    index = similarities_percentages.index(max(similarities_percentages))
    return possible_arrays[index]

In [115]:
possible_matches

[{'card_id': '44d60a41-3f5e-4559-b18c-22c0fe15235e',
  'card_image': array([[254, 254, 254, ..., 254, 254, 254],
         [254, 254, 254, ..., 254, 254, 254],
         [254, 254, 254, ..., 254, 254, 254],
         ...,
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)},
 {'card_id': '6da7cd39-1f8a-4f68-adb7-df2beac02263',
  'card_image': array([[164, 211, 206, ..., 207, 211, 164],
         [211, 254, 254, ..., 254, 254, 212],
         [205, 254, 251, ..., 252, 254, 206],
         ...,
         [205, 254, 251, ..., 252, 255, 206],
         [211, 255, 254, ..., 255, 255, 212],
         [164, 211, 206, ..., 207, 211, 164]], dtype=uint8)},
 {'card_id': '896e5f61-6074-458d-9e8c-aff99fda75d7',
  'card_image': array([[254, 254, 254, ..., 254, 254, 254],
         [254, 254, 254, ..., 254, 254, 254],
         [254, 254, 254, ..., 254, 254, 254],
         ...,
         [255, 255, 255, ..., 255, 

In [121]:
# making the loop to check through all the possible matches and not only one
# THIS ONE IS AN ARRAY OF DICTIONARIES FROM THE API!!!! 
def get_the_matching_picture(image: int) -> int:
    similarities_percentages = []
    for each_possible_match in possible_matches:
        percentage = orb_sim(image, each_possible_match["card_image"])
        similarities_percentages.append(percentage)
    print(similarities_percentages)
    index = similarities_percentages.index(max(similarities_percentages))
    return possible_matches[index]["card_image"]

In [124]:
Image.fromarray(get_the_matching_picture(img1)).show()


[0.25903614457831325, 0.2468354430379747, 0.09090909090909091]


In [94]:
%time similarities_array(img1, possible_arrays)
# Image.fromarray(match).show()

CPU times: total: 1.22 s
Wall time: 896 ms


array([[254, 254, 254, ..., 254, 254, 254],
       [254, 254, 254, ..., 254, 254, 254],
       [254, 254, 254, ..., 254, 254, 254],
       ...,
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)